In [195]:
# imports
# https://github.com/serengil/tensorflow-101/blob/master/python/Matlab-Model-to-Keras.ipynb
# https://sefiks.com/2019/07/15/how-to-convert-matlab-models-to-keras/
import tensorflow as tf
from tensorflow import keras
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from keras import layers
import scipy.io as sio
import os

In [196]:
# prepare data
working_catalog = r"C:\Users\miugi\Downloads"
net_path = os.path.join(working_catalog, "imagenet-vgg-m-1024.mat")
network_mat = sio.loadmat(net_path, matlab_compatible=False, struct_as_record=False)
print(network_mat)
#TODO what about network_mat["meta"]

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Fri Sep 30 08:35:16 2016', '__version__': '1.0', '__globals__': [], 'layers': array([[array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x000001CBB9C68A60>]],
      dtype=object),
        array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x000001CBB9984E20>]],
      dtype=object),
        array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x000001CBB99639D0>]],
      dtype=object),
        array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x000001CBBA09CF40>]],
      dtype=object),
        array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x000001CBB995CAF0>]],
      dtype=object),
        array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x000001CBB9F1C190>]],
      dtype=object),
        array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x000001CBB9F1CA90>]],
      dtype=object),
        array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x000001CBB9F1C6D

In [197]:
ref_model_layers = network_mat['layers'][0]
for layer in ref_model_layers:
    print(layer[0][0].name)

['conv1']
['relu1']
['norm1']
['pool1']
['conv2']
['relu2']
['norm2']
['pool2']
['conv3']
['relu3']
['conv4']
['relu4']
['conv5']
['relu5']
['pool5']
['fc6']
['relu6']
['fc7']
['relu7']
['fc8']
['prob']


In [198]:
for i in range(ref_model_layers.shape[0]):
    ref_model_layer = ref_model_layers[i][0,0].name[0]
    try:
        weights = ref_model_layers[i][0,0].weights[0,0]
        print(ref_model_layer,": ",weights.shape)
    except:
        #print(ref_model_layer)
        print("",end='')

conv1 :  (7, 7, 3, 96)
conv2 :  (5, 5, 96, 256)
conv3 :  (3, 3, 256, 512)
conv4 :  (3, 3, 512, 512)
conv5 :  (3, 3, 512, 512)
fc6 :  (6, 6, 512, 4096)
fc7 :  (1, 1, 4096, 1024)
fc8 :  (1, 1, 1024, 1000)


In [199]:
model = Sequential()
for i in range(num_of_ref_model_layers):
    ref_model_layer = ref_model_layers[i][0][0].name[0]
    if ref_model_layer.find("conv") == 0 or ref_model_layer.find("fc") == 0:
        weights = ref_model_layers[i][0,0].weights
        weights_shape = weights[0][0].shape
        #print(":", weights_shape)
        filter_x = weights_shape[0]; filter_y = weights_shape[1]
        number_of_filters = weights_shape[3]
        
        if ref_model_layer.find("conv") == 0:
            print("ZeroPadding2D((1,1))")
            if i == 0:
                model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
            else:
                model.add(ZeroPadding2D((1,1)))
        
        print("Convolution2D(",number_of_filters,", (",filter_x,", ",filter_y,"), name='",ref_model_layer,"')")
        model.add(Convolution2D(number_of_filters, (filter_x, filter_y), name= ref_model_layer))
        
    else:
        if ref_model_layer.find("relu") == 0:
            print("Activation('relu', name=",ref_model_layer)
            model.add(Activation('relu', name=ref_model_layer))
        elif ref_model_layer.find("dropout") == 0:
            print("Dropout(0.5, name=",ref_model_layer,")")
            model.add(Dropout(0.5, name=ref_model_layer))
        elif ref_model_layer.find("pool") == 0:
            print("MaxPooling2D((2,2), strides=(2,2), name=",ref_model_layer,")")
            model.add(MaxPooling2D((2,2), strides=(2,2), name=ref_model_layer))
        elif ref_model_layer.find("softmax") == 0:
            print("Activation('softmax', name=",ref_model_layer,")")
            model.add(Activation('softmax', name=ref_model_layer))
        else:
            print("-->",ref_model_layer)       

ZeroPadding2D((1,1))
Convolution2D( 96 , ( 7 ,  7 ), name=' conv1 ')
Activation('relu', name= relu1
--> norm1
MaxPooling2D((2,2), strides=(2,2), name= pool1 )
ZeroPadding2D((1,1))
Convolution2D( 256 , ( 5 ,  5 ), name=' conv2 ')
Activation('relu', name= relu2
--> norm2
MaxPooling2D((2,2), strides=(2,2), name= pool2 )
ZeroPadding2D((1,1))
Convolution2D( 512 , ( 3 ,  3 ), name=' conv3 ')
Activation('relu', name= relu3
ZeroPadding2D((1,1))
Convolution2D( 512 , ( 3 ,  3 ), name=' conv4 ')
Activation('relu', name= relu4
ZeroPadding2D((1,1))
Convolution2D( 512 , ( 3 ,  3 ), name=' conv5 ')
Activation('relu', name= relu5
MaxPooling2D((2,2), strides=(2,2), name= pool5 )
Convolution2D( 4096 , ( 6 ,  6 ), name=' fc6 ')
Activation('relu', name= relu6
Convolution2D( 1024 , ( 1 ,  1 ), name=' fc7 ')
Activation('relu', name= relu7
Convolution2D( 1000 , ( 1 ,  1 ), name=' fc8 ')
--> prob


In [200]:
for layer in model.layers:
    layer_name = layer.name
    try:
        print(layer_name,": ",layer.weights[0].shape)
    except:
        print("",end='')

conv1 :  (7, 7, 3, 96)
conv2 :  (5, 5, 96, 256)
conv3 :  (3, 3, 256, 512)
conv4 :  (3, 3, 512, 512)
conv5 :  (3, 3, 512, 512)
fc6 :  (6, 6, 512, 4096)
fc7 :  (1, 1, 4096, 1024)
fc8 :  (1, 1, 1024, 1000)


In [201]:
base_model_layer_names = [layer.name for layer in model.layers]
num_of_ref_model_layers = ref_model_layers.shape[0]

In [202]:
for i in range(num_of_ref_model_layers):
    ref_model_layer = ref_model_layers[i][0,0].name[0]
    if ref_model_layer in base_model_layer_names:
        #we just need to set convolution and fully connected weights
        if ref_model_layer.find("conv") == 0 or ref_model_layer.find("fc") == 0:
            print(i,". ",ref_model_layer)
            base_model_index = base_model_layer_names.index(ref_model_layer)
            
            weights = ref_model_layers[i][0,0].weights[0,0]
            bias = ref_model_layers[i][0,0].weights[0,1]
            
            model.layers[base_model_index].set_weights([weights, bias[:,0]])

0 .  conv1
4 .  conv2
8 .  conv3
10 .  conv4
12 .  conv5
15 .  fc6
17 .  fc7
19 .  fc8


In [203]:
model.save_weights('weights.h5')